<a href="https://colab.research.google.com/github/datakind/hxl-metadata-prediction/blob/main/generate-test-train-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook downloads and prepares training data for predicing Humanitarian Exchange Language (HXL) metadata tags on humanitarian datasets on the Humanitarian Data Exchange (HDX) platform.

This notebook downloads data provided by the HDX team from a google drive folder. The data was captured using an [HXL crawl process](https://github.com/HXLStandard/hdx-hashtag-crawler). It also downloads the [HXL core schema](https://data.humdata.org/dataset/hxl-core-schemas) which defines supported tags and attributes. Finally, the data is split into test and training, being careful to ensure data produced by similar processes and organizations is not present in both test and train.  



# Setup

See [README](README.md) from instructions.

In [ ]:
!pip install gdown==5.2.0
!pip install pandas==2.2.2
!pip install hdx-python-api==6.3.1
!pip install openai==1.35.3

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [13]:
import sys
import os
import requests
import gdown
import tarfile
import pandas as pd
import re
from sklearn.model_selection import train_test_split

from hdx.utilities.easy_logging import setup_logging
from hdx.api.configuration import Configuration
from hdx.data.dataset import Dataset
import hxl
import json
import time
from openai import OpenAI
import numpy as np

if os.getenv("OPENAI_API_KEY") is None:
    from google.colab import userdata

    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
else:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

# If using Colab, this is where Google drive gets mounted. Otherwise leave blank
GOOGLE_BASE_DIR = "/content/drive/MyDrive/Colab"

# Where to save local data files
LOCAL_DATA_DIR = f"{GOOGLE_BASE_DIR}/hxl-metadata-prediction/data/"

# Google drive location of HDXHashtag crawler data. Shared with HDX team
HXL_CRAWLER_DATA_GDRIVE = "https://drive.google.com/file/d/1BDCuh0WVJWK1-1RMC-77cvh4H2Hep_ry/export?format=xlsx"
HXL_CRAWLER_DATA_FILE = LOCAL_DATA_DIR + "/hdx-hxl-output.tgz"

# This is the HXL schema sheet, search HDX to get this link
HXL_SCHEMA_RESOURCE_URL = "https://docs.google.com/spreadsheets/d/1En9FlmM8PrbTWgl3UHPF_MXnJ6ziVZFhBbojSJzBdLI/export?format=xlsx"
HXL_SCHEMA_LOCAL_FILE = LOCAL_DATA_DIR + "/hxl-core-schema.xlsx"

# Number of records in data excerpts
DATA_EXCERPT_SIZE = 10

# Data Summary LLM, used to summarize tabular data for use in prompts
# DATA_SUMMARY_LLM = "gpt-4o-mini"
DATA_SUMMARY_LLM = "gpt-3.5-turbo"

# Set this to a name of your choosing
HDX_USER_AGENT = "hxl-metadata-prediction"

input_options = hxl.input.InputOptions(http_headers={"User-Agent": HDX_USER_AGENT})

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)

## Get HDX connection

We need a connection to HDX to extract excerpts of data for prompts.

In [14]:
def setup_hdx_connection(agent_name):
    """
    Set up the HDX connection with the specified agent name.

    Parameters:
    - agent_name (str): The name of the agent.

    Returns:
    None
    """
    try:
        Configuration.create(hdx_site="prod", user_agent=agent_name, hdx_read_only=True)
    except:
        print("Configuration already created, continuing ...")


# Note, if you run this twice you will get a 'Configuration already exists!' error, but it can be ignored
setup_hdx_connection(HDX_USER_AGENT)

Configuration already created, continuing ...


## Download HXL Core schema

We will download the original google sheet defining the HXL core schema, as found on [HDX](https://data.humdata.org/dataset/hxl-core-schemas).





In [15]:
gdown.download(HXL_SCHEMA_RESOURCE_URL, HXL_SCHEMA_LOCAL_FILE, quiet=False, fuzzy=True)

response = requests.get(HXL_SCHEMA_RESOURCE_URL)
with open(HXL_SCHEMA_LOCAL_FILE, "wb") as f:
    f.write(response.content)

df = pd.read_excel(HXL_SCHEMA_LOCAL_FILE, sheet_name="Core hashtags")
hashtags_list = df["Hashtag"][1:].tolist()

df = pd.read_excel(HXL_SCHEMA_LOCAL_FILE, sheet_name="Core attributes")
attributes_list = df["Attribute"][1:].tolist()

# Remove rows with disallowed tags or attributes
APPROVED_HXL_SCHEMA = hashtags_list + attributes_list

print("Approved HXL schema ...")
print(APPROVED_HXL_SCHEMA)

Downloading...
From: https://docs.google.com/spreadsheets/d/1En9FlmM8PrbTWgl3UHPF_MXnJ6ziVZFhBbojSJzBdLI/export?format=xlsx
To: /content/drive/MyDrive/Colab/hxl-metadata-prediction/data/hxl-core-schema.xlsx
240kB [00:00, 29.9MB/s]


Approved HXL schema ...
['#access', '#activity', '#adm1', '#adm2', '#adm3', '#adm4', '#adm5', '#affected', '#beneficiary', '#capacity', '#cause', '#channel', '#contact', '#country', '#crisis', '#currency', '#date', '#delivery', '#description', '#event', '#frequency', '#geo', '#group', '#impact', '#indicator', '#inneed', '#item', '#loc', '#meta', '#modality', '#need', '#operations', '#org', '#output', '#population', '#reached', '#region', '#respondee', '#sector', '#service', '#severity', '#status', '#subsector', '#targeted', '#value', '+abducted', '+acronym', '+activity', '+adolescents', '+adults', '+approved', '+ar', '+bounds', '+budget', '+canceled', '+children', '+cluster', '+code', '+converted', '+coord', '+dest', '+displaced', '+elderly', '+elevation', '+email', '+en', '+end', '+es', '+f', '+fa', '+fr', '+funder', '+hh', '+i', '+id', '+idps', '+impl', '+incamp', '+ind', '+infants', '+infected', '+injured', '+killed', '+label', '+lat', '+lon', '+m', '+ms', '+name', '+noncamp', '+num

# Analysis

## Download HXL crawler data

This data was generated using the [HDX Hashtag Crawler](https://github.com/dividor/hdx-hashtag-crawler) over several days and saved to Google drive.

In [16]:
gdown.download(HXL_CRAWLER_DATA_GDRIVE, HXL_CRAWLER_DATA_FILE, quiet=False, fuzzy=True)
print(f"HXL core schema saved to {HXL_CRAWLER_DATA_FILE}")

Downloading...
From: https://drive.google.com/file/d/1BDCuh0WVJWK1-1RMC-77cvh4H2Hep_ry/export?format=xlsx
To: /content/drive/MyDrive/Colab/hxl-metadata-prediction/data/hdx-hxl-output.tgz
3.04kB [00:00, 1.38MB/s]

HXL core schema saved to /content/drive/MyDrive/Colab/hxl-metadata-prediction/data//hdx-hxl-output.tgz


## Identify Unique combinations of HXL tags we want for training data

The HXL crawler and reports data create a hash based on column names, referred to as a 'Resource pattern' by the HDX team. This is useful because a lot of HDX resources (tables) are generated by automatic pipeline, so rather than duplicating very similar data in our training set, we will focus on unique combinations of column headers. This is an indirect way of balancing the data.

In [17]:
df = pd.read_csv(LOCAL_DATA_DIR + "/output/hdx-expanded-hashed-stats.csv")

# We'll keep one row per column, Hashtag with Attributes has what we require
df.drop(columns=["Attribute", "Hashtag"], inplace=True)
df.drop_duplicates(inplace=True)

# Remove HXL tags row in the metadata (we keep them for actual data)
df = df[1:]

print("Number of rows in data ...")
print(df.shape)

print("Unique data providers ...")
print(len(df["Data provider"].unique()))

print("Unique HDX resource ids ...")
print(len(df["HDX resource id"].unique()))

display(df.head())

<ipython-input-17-194518c6290b>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(LOCAL_DATA_DIR + "/output/hdx-expanded-hashed-stats.csv")


Number of rows in data ...
(487297, 9)
Unique data providers ...
120
Unique HDX resource ids ...
43074


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash
1,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
2,#affected+idp+ind,Total IDP IND,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
4,#affected+idp+male,Total IDP Male Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
6,#affected+female+idp,Total IDP Female Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
8,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c


Let's use the column hash created by the crawler to find unique combinations of tags

In [ ]:
hash_count = df.groupby("Hash").size()


In [19]:
hash_resources = (
    df.groupby("Hash")["HDX resource id"].nunique().sort_values(ascending=False)
)

for col in ["HDX resource id", "HDX dataset id", "Data provider"]:
    hash_resources = hash_resources.reset_index()
    hash_resources[f"Unique {col}"] = hash_resources["Hash"].map(
        df.groupby("Hash")[col].unique()
    )

display(hash_resources)

hash_resources.to_excel(f"{LOCAL_DATA_DIR}/hxl_hash_resources.xlsx", index=False)

,level_0,index,Hash,HDX resource id,Unique HDX resource id,Unique HDX dataset id,Unique Data provider
0,0,0,0x102125f4dd16c64,13858,"[51b2e4ec-aca5-4b97-bbb7-c005175b682e, a6ef8040-3b15-47ae-9973-1dbc113673cf, f579cf0e-5535-4414-897f-2f8c05105180, 66c62464-017b-4aa6-845f-9ec2487acb82, 91e1cb98-353b-487e-a14d-b0eea783da6f, 65f38...","[who-data-for-south-sudan, who-data-for-montenegro, who-data-for-zimbabwe, who-data-for-zambia, who-data-for-yemen, who-data-for-viet-nam, who-data-for-venezuela-bolivarian-republic-of, who-data-f...",[world-health-organization]
1,1,1,0x428a8e37940223d8,9328,"[2e130cdf-c850-4533-b2f3-e961adbec48a, 9e160d82-691d-49a6-979b-0ff0dbb6b7a8, 002501bc-7efb-4335-b672-d045cd76bc5b, 0d0e0fc4-e4f1-49cd-ad30-42cc8dc08b74, f2c1ea93-d241-413b-8140-c009da88d912, 23e36...","[world-bank-combined-indicators-for-zimbabwe, world-bank-trade-indicators-for-zimbabwe, world-bank-external-debt-indicators-for-zimbabwe, world-bank-climate-change-indicators-for-zimbabwe, world-b...",[world-bank-group]
2,2,2,0x31fda1ef985b4a59,3425,"[9c751883-698a-4a2c-9475-ff828e9c11db, 791b69af-df57-4157-96c0-c0d8d308315e, b055f1f7-8cdc-4ff8-a412-9f54d6e56c41, 6ae8568f-449b-4b59-9ad2-79d7df94cd9c, 4f4c7462-6b13-4125-a88a-f8e9ac0c837b, abb79...","[dhs-data-for-sao-tome-and-principe, dhs-data-for-rwanda, dhs-data-for-philippines, dhs-data-for-peru, dhs-data-for-paraguay, dhs-data-for-papua-new-guinea, dhs-data-for-pakistan, dhs-data-for-nig...",[dhs]
3,3,3,0x19598575d3397e19,3232,"[ed7b5bd2-7818-4d7a-9ff0-8ba0d97bf7d5, 8f239e93-76c0-4287-a414-3d17a5e55344, 040efd19-3d71-4e0b-8939-f6b46c465868, 8db41d1c-af8f-4f26-9d40-5b80b1bf72e8, 6482dc75-0e79-40d9-aacf-cdcde3c368a6, c9981...","[dhs-subnational-data-for-sao-tome-and-principe, dhs-subnational-data-for-rwanda, dhs-subnational-data-for-philippines, dhs-subnational-data-for-peru, dhs-subnational-data-for-paraguay, dhs-subnat...",[dhs]
4,4,4,0x16d2b679132fea10,2147,"[295cd9e4-8464-43ee-ad17-47196991a1f7, 34337d16-017d-4d69-834c-a5e0fc21a549, b51b8c0e-494d-488b-98d5-a70fd9451b90, 75076d6a-8d3f-49e3-b4f4-7c889bc82806, 7e0b2b37-73b8-4c69-bb11-6ba954fa0cd9, 08ea0...","[unhcr-population-data-for-world, unhcr-population-data-for-zwe, unhcr-population-data-for-zmb, unhcr-population-data-for-zaf, unhcr-population-data-for-yem, unhcr-population-data-for-wsm, unhcr-p...",[unhcr]
...,...,...,...,...,...,...,...
639,639,639,0x22def8e1b7b0c742,1,[68328f42-9276-423e-80d0-fe89630804ff],[3w-december-2017],[ocha-ethiopia]
640,640,640,0x4b7012601f2de402,1,[63143067-46e0-4fb5-b131-d83ee45122ab],[ethiopia-settlements],[ocha-ethiopia]
641,641,641,0x4aefd35864aaa20a,1,[6ddcfbc9-fa06-4b14-b9a4-ce96d3fae65e],[base-acceso-internet-personas-entre-los-5-y-19-anos-2018],[immap]
642,642,642,0x4acf4e36d67d877,1,[903326f2-b372-4786-9973-87226cb15e41],[people-in-need-2008-2019],[ocha-fts]


In [20]:
# Extract a single resource_id for each hash
resource_ids = hash_resources["Unique HDX resource id"].apply(lambda x: x[0])
print(resource_ids)

0      51b2e4ec-aca5-4b97-bbb7-c005175b682e
1      2e130cdf-c850-4533-b2f3-e961adbec48a
2      9c751883-698a-4a2c-9475-ff828e9c11db
3      ed7b5bd2-7818-4d7a-9ff0-8ba0d97bf7d5
4      295cd9e4-8464-43ee-ad17-47196991a1f7
                       ...                 
639    68328f42-9276-423e-80d0-fe89630804ff
640    63143067-46e0-4fb5-b131-d83ee45122ab
641    6ddcfbc9-fa06-4b14-b9a4-ce96d3fae65e
642    903326f2-b372-4786-9973-87226cb15e41
643    8b0feea6-20e4-45dc-aaae-c8b6fbd5a9f4
Name: Unique HDX resource id, Length: 644, dtype: object


In [21]:
df_subset = df[df["HDX resource id"].isin(resource_ids)]

print("Column data subset to one resource ID per hash ...")
print(df_subset.shape)

print("Unique data providers ...")
print(len(df_subset["Data provider"].unique()))

print("Unique HDX resource ids ...")
print(len(df_subset["HDX resource id"].unique()))

Column data subset to one resource ID per hash ...
(7834, 9)
Unique data providers ...
119
Unique HDX resource ids ...
644


## Remove unsupported HXL tags and attributes

Next, we will remove HXL tags and attributes which are not officially supported in the HXL Core schema.

In [22]:
def filter_for_schema(text):
    """
    Filters the input text to remove spaces and extract approved HXL schema tokens.

    Args:
        text (str): The input text to be filtered.

    Returns:
        str: The filtered text containing only approved HXL schema tokens.

    """
    # print(f"Tokens before: {text}")
    if " " in text:
        text = text.replace(" ", "")

    tokens_raw = text.split("+")
    tokens = [tokens_raw[0]]
    for t in tokens_raw[1:]:
        tokens.append(f"+{t}")

    filtered = []
    for t in tokens:
        if t in APPROVED_HXL_SCHEMA:
            if t not in filtered:
                filtered.append(t)
    filtered = "".join(filtered)

    if len(filtered) > 0 and filtered[0] != "#":
        filtered = ""

    return filtered


def filter_disallowed_hxl(column_data, hxl_col="Hashtag with Attributes"):
    """
    Filter the column_data DataFrame based on a specific condition.

    Args:
        column_data (pd.DataFrame): The DataFrame containing the column data.
        hxl_col (str, optional): The name of the column to filter on. Defaults to 'Hashtag with Attributes'.

    Returns:
        pd.DataFrame: The filtered DataFrame containing the allowed rows.
        pd.DataFrame: The filtered DataFrame containing the disallowed rows.
    """
    print("Before", column_data.shape)
    allowed = []
    disallowed = []
    for index, row in column_data.iterrows():
        if row[hxl_col] == filter_for_schema(row[hxl_col]):
            allowed.append(row)
        else:
            disallowed.append(row)
    allowed = pd.DataFrame(allowed)
    disallowed = pd.DataFrame(disallowed)
    print("After", allowed.shape)
    return allowed, disallowed


data, disallowed = filter_disallowed_hxl(df_subset)
print(data.shape)

display(disallowed)

Before (7834, 9)
After (3777, 9)
(3777, 9)


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash
2,#affected+idp+ind,Total IDP IND,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
4,#affected+idp+male,Total IDP Male Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
6,#affected+female+idp,Total IDP Female Ind,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c
256,#meta+appeal+type,atype,SSD,ifrc,ifrc-appeals-data-for-south-sudan,4110b824-3338-453f-ae5d-89ca80f5b147,2023-03-13,true,0x1d1434ee319a1be
260,#meta+appeal+id,code,SSD,ifrc,ifrc-appeals-data-for-south-sudan,4110b824-3338-453f-ae5d-89ca80f5b147,2023-03-13,true,0x1d1434ee319a1be
...,...,...,...,...,...,...,...,...,...
689570,#lat_deg,prevlat,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce
689571,#lon_deg,prevlon,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce
689572,#period_date,datelabel,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce
689573,#x_time,hours,VUT,brcmapsteam,cyclone-pam-path,a8ccd9d2-8328-487a-b04b-ca3f3f2e0ea3,2015-03-16,True,0x1d4a8deeb40f76ce


In [23]:
disallowed_sample = disallowed.sample(20)
display(disallowed_sample["Hashtag with Attributes"])

,Hashtag with Attributes
653446,#admin1+code
689561,#period_date
632530,#indicator+num+student
611944,#value+cerf+funding+gm0+total+usd
687700,#affected+date+w5
653131,#value+sanitation5
95582,#indicator+educationfacility+entry+num
600075,#loc+name+port
612658,#affected+ind+migrant
689562,#x_time


## Download data excerpts

Using our subset of resource_ids for each hash, extract column data excerpts.

In [24]:
def populate_data_excerpts(df_in):
    """
    Populates the 'Data excerpt' column in the given DataFrame with data excerpts from each resource's columns.

    Args:
        df_in (pandas.DataFrame): The input DataFrame containing the data.

    Returns:
        pandas.DataFrame: The modified DataFrame with the 'Data excerpt' column populated.
    """

    df = df_in.copy()

    df["Data excerpt"] = ""

    datasets_resources = df[["HDX dataset id", "HDX resource id"]].drop_duplicates()
    datasets_resources.reset_index(drop=True, inplace=True)

    print("For each resource, extract a data excerpt for each column ...")

    num_rows = datasets_resources.shape[0]
    for index, row in datasets_resources.iterrows():

        if index % 10 == 0:
            print(
                f"Processing {index} of {num_rows} ({(index/num_rows)*100:.2f}%) resources"
            )

        dataset_id = row["HDX dataset id"]
        resource_id = row["HDX resource id"]
        try:
            dataset = Dataset.read_from_hdx(dataset_id)
            if dataset is None:
                print(f"Dataset {dataset_id} not found!")
                continue
        except Exception as e:
            print(f"Error reading dataset {dataset_id} ... {e}")
            continue

        resources = dataset.get_resources()
        for resource in resources:
            if resource["id"] == resource_id:
                print(
                    f"    Accessing data for resource {resource_id}, {resource['name']}"
                )
                try:
                    url, path = resource.download(LOCAL_DATA_DIR)
                    df.loc[df["HDX resource id"] == resource_id, "File"] = path
                    df.loc[df["HDX resource id"] == resource_id, "URL"] = url

                    with hxl.data(resource["url"], input_options) as source:
                        columns = [column.header for column in source.columns]
                        tags = [
                            column.get_display_tag(sort_attributes=True)
                            for column in source.columns
                        ]
                        data = {}
                        rowcount = 0
                        for row in source:
                            if rowcount > DATA_EXCERPT_SIZE:
                                break
                            i = 0
                            for colvalue in row:
                                colname = columns[i]
                                if colname not in data:
                                    data[colname] = [colvalue]
                                else:
                                    data[colname].append(colvalue)
                                i += 1
                            rowcount += 1

                        for col in columns:
                            if col in data:
                                # print(f"       Setting data excerpt for column {col} >> {data[col]} ...")
                                df.loc[
                                    (df["HDX resource id"] == resource_id)
                                    & (df["Text header"] == col),
                                    "Data excerpt",
                                ] = str(data[col])

                except Exception as e:
                    print(
                        f"Error accessing data for resource {resource_id}, {resource['name']} ... {e}"
                    )

    return df


df = data.copy()
df = populate_data_excerpts(df)
display(df)

df.to_csv(f"{LOCAL_DATA_DIR}/hxl_hash_resources_data.csv", index=False)

print(data.shape)
print(df.shape)

data = df.copy()

For each resource, extract a data excerpt for each column ...
Processing 0 of 612 (0.00%) resources
    Accessing data for resource 26ecc26f-74e7-46af-b450-8872dca0b63b, DRC - Baseline Assessment - M23 Crisis 13 - February 2024
    Accessing data for resource dbf9b4bd-1321-4846-b6f0-4654509d3626, admin1-summaries-earthquake.csv
    Accessing data for resource 4110b824-3338-453f-ae5d-89ca80f5b147, IFRC Appeals Data for South Sudan
    Accessing data for resource b4e5634e-42e0-4ca5-8893-8e15cad6b620, fts_requirements_funding_pse.csv
    Accessing data for resource 4854ed70-56fb-4792-8845-bdeb51418c66, fts_incoming_funding_vnm.csv
    Accessing data for resource 7b373ee7-af33-4140-92a0-83d2693eb74c, CERF Donor Contributions (HXLated).csv
    Accessing data for resource 636c8890-8747-442b-87c9-895133314dcb, idmc event data for WSM
    Accessing data for resource 3a53c43d-83bc-46e7-b66a-38ccca4b536c, AWSD_SD_security_incidents.csv
    Accessing data for resource 0908c976-8b62-4a7e-9a38-9750

ERROR:hxl.REMOTE_ACCESS:2024-07-31 14:59:19 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=projectid
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: projectid
ERROR:hxl.REMOTE_ACCESS:2024-07-31 14:59:19 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=projectid
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: projectid


    Accessing data for resource 697c656a-684c-4ae1-9849-0db4d6b732ad, DTM Yemen Displacement Tracking  11 - 17 February 2024
    Accessing data for resource 2ca8e86c-461c-482f-ac5e-9cdb8871a4ad, Lebanon - IDP Tracking Dataset - ROUND 24- 15-02-2024 - HDX.xlsx
    Accessing data for resource c5ce40d6-07b1-4f36-955a-d6196436ff6b, EMDAT-country-profiles_2024_07_29.xlsx
    Accessing data for resource a1a73231-7a96-4501-9675-97cddf3c3893, Suite of Food Security Indicators for Zimbabwe
    Accessing data for resource 26e65696-95c3-4eac-8d18-c150526e34e0, Deflators data for Zimbabwe
    Accessing data for resource b81e33f7-8942-4113-b675-8fbffc8a714f, Exchange rates data for Holy See
    Accessing data for resource 6f659727-e3d8-4eef-bc2b-a3a836b65243, wpdx_water_points_hti
Processing 30 of 612 (4.90%) resources
    Accessing data for resource 6926dff7-658a-49e1-8d61-0ed8a983fbe1, ipc_global_national_long_latest.csv
    Accessing data for resource a6f9f9b8-d265-4099-9ead-5e65fd26c074, Mozamb

ERROR:hxl.REMOTE_ACCESS:2024-07-31 15:00:12 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=name, #reporting+or+implementing+org?
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: name, #reporting+or+implementing+org?
ERROR:hxl.REMOTE_ACCESS:2024-07-31 15:00:12 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=name, #reporting+or+implementing+org?
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: name, #reporting+or+implementing+org?


    Accessing data for resource e77f3400-5d43-4a46-81a9-bf86e8958141, InterAction Member Data for Zimbabwe


...

    Accessing data for resource 3106b05f-e171-49ac-a353-d7b56aed918f, Lake_Chad_Basin_Appeal_Status_2017-02-27.csv
    Accessing data for resource 1aaa5281-de4f-4ffa-a5ef-37c754a86478, Lake_Chad_Basin_Displaced_2017-02-23.csv
    Accessing data for resource 020000e5-e7aa-4027-b58a-efe7396ff32e, Lake_Chad_Basin_Estimated_Population_2017-02-23.csv
    Accessing data for resource 48eea6e6-04d9-4c6a-ad38-80ff8365f601, 201612---Pakistan-4Ws-KP-FATA-2016-HXL (3).xlsx
    Accessing data for resource 218ab806-0651-451a-a731-e0ccb80fe1d9, 06_09_update zika_global cases_IFRC.xlsx
Processing 600 of 612 (98.04%) resources
    Accessing data for resource 968202f1-856a-4906-ae87-c730e9b1dd27, PHL_haima_houses_damaged_pcoded_ndrrmc_sitrep_9_20161025.csv
    Accessing data for resource 7211bf25-c9a1-4496-9bbb-a3e10efb2e4f, 15_09_zika_cases_export_Zika_Team.xlsx
    Accessing data for resource 920df0fe-411d-47ee-8004-11b2302fbe1c, 160909_5W 2.0_HDX.xlsx
    Accessing data for resource ac8d491f-338f-414

ERROR:hxl.REMOTE_ACCESS:2024-07-31 15:57:54 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=#sector#subsector
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: #sector#subsector
ERROR:hxl.REMOTE_ACCESS:2024-07-31 15:57:54 [error    ] Skipping column(s) with malformed hashtag specs function=parse_list hastags=#sector#subsector
ERROR:hxl.model:Skipping column(s) with malformed hashtag specs: #sector#subsector


    Accessing data for resource 58069fa8-18b6-4132-a242-32216815a398, Ecuador Earthquake - April 2016 - Severity index
    Accessing data for resource 5c3a03d2-53de-4754-86b3-c7116d927155, Burundi 3W_formated.xlsx
    Accessing data for resource 62d4b11f-1a29-4753-a54f-c327f21f0b0b, 3W - 5 May - Final consolidated.xlsx
    Accessing data for resource f7daccff-bcc7-41cd-bee0-4133ce6b8988, Somalia NGO Consortium Data.xlsx
    Accessing data for resource 9cba28b4-d112-4850-96b4-81122af45f9b, 141121 LR Health Care Facilities.xlsx
Processing 610 of 612 (99.67%) resources
    Accessing data for resource f78dc606-04e2-4fb6-a7eb-9eb995c33f76, 1501 Sierra Leone Health Centers.xlsx
    Accessing data for resource 5d2531d6-c03a-449b-afdd-52c07d687679, Guinea health-facility master data


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash,Data excerpt,File,URL
1,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c,[319283],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
8,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c,[587705],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx
100,#country,country_name,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv
101,#adm1+name,region_name,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,,/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv
102,#geo+lat,latitude,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['34.5527', '34.9568', '34.9619', '34.3033', '34.0121', '34.2743', '34.7693', '35.4474', '35.8025', '34.8046', '33.3211']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv
...,...,...,...,...,...,...,...,...,...,...,...,...
689531,#adm1,Province,SLE,standby-task-force,141121-sierra-leone-health-facilities,f78dc606-04e2-4fb6-a7eb-9eb995c33f76,2014-11-01,True,0x5c648c638f567754,"['Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern', 'Eastern']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/1501 Sierra Leone Health Centers2.xlsx,https://data.humdata.org/dataset/7453fb80-752b-4078-a892-d936f9846dab/resource/f78dc606-04e2-4fb6-a7eb-9eb995c33f76/download/1501-sierra-leone-health-centers.xlsx
689533,#adm2,District,SLE,standby-task-force,141121-sierra-leone-health-facilities,f78dc606-04e2-4fb6-a7eb-9eb995c33f76,2014-11-01,True,0x5c648c638f567754,"['Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema', 'Kenema']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/1501 Sierra Leone Health Centers2.xlsx,https://data.humdata.org/dataset

(3777, 9)
(3777, 12)


## Data cleaning

In [25]:
data = data[data["Data excerpt"].notnull()]
data = data[data["Data excerpt"].str.contains(r"[A-Za-z0-9]")]

print(data.shape)

(3341, 12)


## Tags distribution

In [26]:
data["tag"] = data["Hashtag with Attributes"].apply(lambda x: x.split("+")[0])
tag_counts_train = data["tag"].value_counts()

print(tag_counts_train)

tag
#adm1           523
#adm2           473
#affected       380
#country        284
#date           244
#org            228
#adm3           188
#inneed         140
#sector         109
#geo            106
#targeted        75
#loc             67
#activity        67
#status          59
#population      57
#indicator       55
#region          49
#meta            42
#reached         32
#adm4            32
#subsector       13
#event           13
#beneficiary     12
#cause           12
#value           11
#item            10
#severity         9
#output           8
#crisis           6
#currency         4
#service          4
#adm5             4
#contact          4
#access           4
#capacity         3
#impact           3
#description      3
#frequency        2
#group            2
#modality         2
#delivery         1
#operations       1
Name: count, dtype: int64


## Generate Data table descriptions using LLM
Though there are descriptions on HDX for datasets, there aren't always detailed descriptions for tables in CSV and Excel files. We will generate these using GPT-3.5-Turbo to include in the prompt as they provide valuable context when considering what HXL tags apply to a column

In [27]:
def generate_data_description(data, file_name):
    """
    Generate a short description of a dataset based on a summary of its content.

    Args:
        data (DataFrame): The input dataset for which a description needs to be generated.
        file_name (str): The name of the file.

    Returns:
        str: A short description of the dataset.
    """

    prompt = f"""
      This data file ...

      {file_name}

      Has data that looks like this ...

      {data.iloc[0:DATA_EXCERPT_SIZE].to_string()}

      Summarize this dataset
    """

    # Define conversation messages
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant Summarizing data into one paragraph",
        },
        {"role": "user", "content": prompt},
    ]

    # Request a completion (description) from the OpenAI API
    try:
        response = client.chat.completions.create(
            model=DATA_SUMMARY_LLM,
            messages=messages,
            temperature=0,
            max_tokens=300,
            stop=["\n\n"],
        )
    except Exception as e:
        print(f"Error generating description for {file_name} ... {e}")
        return ""

    return response.choices[0].message.content


def generate_data_descriptions(data_in):

    data = data_in.copy()
    data["Data description"] = ""

    unique_resources = data["HDX resource id"].unique().shape[0]
    print(f"\n\nUnique resources: {unique_resources}\n")

    count = 0
    dataset_descriptions = {}
    for index, row in data.iterrows():
        resource_name = row["File"].replace(f"{LOCAL_DATA_DIR}/", "")
        resource_id = row["HDX resource id"]
        if resource_id not in dataset_descriptions:
            # We need a try/except as not all downloads from HDX succeed
            try:
                if ".xlsx" in resource_name:
                    df = pd.read_excel(resource_name)
                elif ".csv" in resource_name:
                    df = pd.read_csv(resource_name)
                else:
                    print(f"Unknown file type for {resource_name}")
                    continue
            except Exception as e:
                print(f"Error reading {resource_name} ... {e}")
                continue

            dataset_descriptions[resource_id] = generate_data_description(
                df, resource_name
            )

            print(f"Description: {dataset_descriptions[resource_id]}")

            data.loc[data["HDX resource id"] == resource_id, "Data description"] = (
                dataset_descriptions[resource_id]
            )

            count += 1
            if count % 10 == 0:
                print(f"Processed {round(count/unique_resources,2)*100}% resources ...")

    return data


df = generate_data_descriptions(data)
display(df)

df.to_csv(f"{LOCAL_DATA_DIR}/hxl_hash_resources_data.csv", index=False)



Unique resources: 555

Description: The dataset from the file "DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx" contains information on the total number of internally displaced persons (IDPs) and returnees in the Democratic Republic of Congo. The data includes the total number of IDP households, individual IDPs, male and female IDPs, and returnees. For instance, there are 319,283 IDP households, 1,548,732 individual IDPs, 646,805 male IDPs, 901,927 female IDPs, and 587,705 returnees.
Description: The dataset contains earthquake data for various administrative regions within Afghanistan, including information such as country name, admin1 name, latitude, longitude, aggregation method, indicator name, and indicator value. Each row represents a different administrative region with corresponding earthquake magnitude values. The dataset appears to be structured with columns for specific data attributes and rows for individual administrative regions, providing a comprehensiv

<ipython-input-27-444004c988ae>:66: DtypeWarning: Columns (0,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(resource_name)


Description: The dataset contains information on demographics and locations of forcibly displaced and stateless persons globally. It includes data on the year, country of origin and asylum, population type, location, urban/rural classification, accommodation type, and demographic breakdown by age and gender. The data provides details on the number of affected individuals in different categories such as infants, children, adolescents, adults, and elderly, as well as unknown age groups. The dataset covers various countries and years, with total population figures included.
Description: The dataset contains information on projects in Ethiopia for December 2023, including details such as reporting month, project status, donor organizations, implementing agencies, locations at regional, zonal, and woreda levels, sector clusters, activities, and geographical coordinates. The data shows ongoing projects primarily focused on community participation and self-governance in various locations with

<ipython-input-27-444004c988ae>:66: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(resource_name)


Description: The dataset global_pcodes3.csv contains information on locations, administrative levels, P-Codes, names, parent P-Codes, and valid from dates. The data includes details such as country codes, admin levels, specific P-Codes for different regions within Afghanistan, corresponding names of these regions, parent P-Codes, and the date from which the data is valid. The dataset seems to focus on administrative divisions within Afghanistan, providing a structured format for organizing and referencing geographical information.
Description: The dataset contains information on different clusters, organizations, organization types, regions, and districts. The data includes columns such as Cluster, Organization, Org Type, Region, and District. The dataset appears to list various organizations operating within different regions and districts, with each entry specifying the cluster to which the organization belongs, the organization's name, type, and the specific region and district wher

...

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Description: The dataset contains information on various organizations, sectors, types of activities, codes, quantities, units, locations (including provinces, cantons, and parishes), types of locations, shelters, status of activities, and total number of beneficiaries. The data includes details such as organization names, sectors like Water, Sanitation, and Hygiene, specific activities undertaken by each organization, quantities of items distributed or actions taken, and the status of each activity (e.g., in execution or finalized). The dataset covers multiple entries for different organizations, each with specific details related to their humanitarian efforts in Ecuador.
Unknown file type for /content/drive/MyDrive/Colab/hxl-metadata-prediction/data/Ecuador Earthquake - April 2016 - Severity index2.google sheet
Unknown file type for /content/drive/MyDrive/Colab/hxl-metadata-prediction/data/Ecuador Earthquake - April 2016 - Severity index2.google sheet
Unknown file type for /content/d

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Description: The dataset is stored in an Excel file named "141121 LR Health Care Facilities2.xlsx" and contains information related to health care facilities in Guinea, Liberia, and Sierra Leone. The data was collected, collated, and cleaned by Standby Task Force volunteers during September and October 2014. The dataset includes links to updated maps and is free for use in nonprofit humanitarian projects. The file also provides a link to the Standby Task Force Maps portal for further information.
Description: The dataset contains information on health centers in Sierra Leone, including details such as center ID, status, date opened, type of center, activity, location coordinates, address, and source of information. The data includes various health centers across different districts and chiefdoms in Sierra Leone, with details on their capacities, equipment, and verification status. The dataset seems to focus on health posts, health centers, and villages in Sierra Leone, providing inform

,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash,Data excerpt,File,URL,tag,Data description
1,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c,[319283],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx,#affected,"The dataset from the file ""DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx"" contains information on the total number of internally displaced persons (IDPs) and returnees in the Dem..."
8,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,true,0x2cc7fd3129c0d18c,[587705],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx,#affected,"The dataset from the file ""DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx"" contains information on the total number of internally displaced persons (IDPs) and returnees in the Dem..."
100,#country,country_name,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#country,"The dataset contains earthquake data for various administrative regions within Afghanistan, including information such as country name, admin1 name, latitude, longitude, aggregation method, indica..."
102,#geo+lat,latitude,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['34.5527', '34.9568', '34.9619', '34.3033', '34.0121', '34.2743', '34.7693', '35.4474', '35.8025', '34.8046', '33.3211']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#geo,"The dataset contains earthquake data for various administrative regions within Afghanistan, including information such as country name, admin1 name, latitude, longitude, aggregation method, indica..."
103,#geo+lon,longitude,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['69.3376', '69.6258', '68.887', '68.2174', '69.1631', '70.4529', '70.1638', '69.798', '68.9114', '67.2373', '67.812']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#geo,

## Train/Test split

In this section we will create train and test datasets for fine tuning.

In [28]:
data = pd.read_csv(f"{LOCAL_DATA_DIR}/hxl_hash_resources_data.csv")
print(data.shape)
display(data)

(3341, 14)


,Hashtag with Attributes,Text header,Locations,Data provider,HDX dataset id,HDX resource id,Date created,Unnamed: 9,Hash,Data excerpt,File,URL,tag,Data description
0,#affected+hh,Total IDP HH,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[319283],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx,#affected,"The dataset from the file ""DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx"" contains information on the total number of internally displaced persons (IDPs) and returnees in the Dem..."
1,#affected+ind+returnees,Total Returnees,COD,international-organization-for-migration,drc-displacement-idps-returnees-m23-crisis-north-kivu-province-baseline-assessment-iom-dtm,26ecc26f-74e7-46af-b450-8872dca0b63b,2023-10-16,True,0x2cc7fd3129c0d18c,[587705],/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx,https://data.humdata.org/dataset/3554c498-660a-45cb-ada5-86a1fbcd6056/resource/26ecc26f-74e7-46af-b450-8872dca0b63b/download/adc_27jan-12_feb_update_public_v2.xlsx,#affected,"The dataset from the file ""DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx"" contains information on the total number of internally displaced persons (IDPs) and returnees in the Dem..."
2,#country,country_name,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#country,"The dataset contains earthquake data for various administrative regions within Afghanistan, including information such as country name, admin1 name, latitude, longitude, aggregation method, indica..."
3,#geo+lat,latitude,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['34.5527', '34.9568', '34.9619', '34.3033', '34.0121', '34.2743', '34.7693', '35.4474', '35.8025', '34.8046', '33.3211']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#geo,"The dataset contains earthquake data for various administrative regions within Afghanistan, including information such as country name, admin1 name, latitude, longitude, aggregation method, indica..."
4,#geo+lon,longitude,AFG BFA BDI CMR CAF TCD COL COD ETH HTI MLI MOZ MMR NER NGA SOM SSD PSE SDN SYR UKR VEN YEM,eth-zurich-weather-and-climate-risks,climada-earthquake-dataset,dbf9b4bd-1321-4846-b6f0-4654509d3626,2024-02-23,NaN,0x234606b299c1e43e,"['69.3376', '69.6258', '68.887', '68.2174', '69.1631', '70.4529', '70.1638', '69.798', '68.9114', '67.2373', '67.812']",/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/admin1-summaries-earthquake8.csv,https://data.humdata.org/dataset/744f4f0b-3172-4397-9609-5ec0b9d34fcb/resource/dbf9b4bd-1321-4846-b6f0-4654509d3626/download/admin1-summaries-earthquake.csv,#geo,"The d

###  Split by data provider organization

On HDX, the hierarchy is ...

Organization > datasets > resources > tables

A random train/test split will result in data from files in a dataset being in both train and test, which would pollute the test set with very similar data to training. So we will split by organization

In [68]:
def split_data(column_data, provider_col, test_size=0.2, random_state=42):
    """
    Perform train-test split on datasets, print information, and return X_train and X_test.

    The split is done by organizations, to try and avoid the situation where an org provides
    similar data files. Also, we exclude orgs which are subsidiaries from the test set, eg ocha-*
    as presumably each subsid will provide similar data. The aim is that the test set is new.

    Parameters:
    - column_data (pd.DataFrame): DataFrame containing column data.
    - provider_col (string): Name of column holding data providers.
    - test_size (float): The proportion of the dataset to include in the test split.
    - random_state (int): Seed for random number generation.

    Returns:
    - pd.DataFrame, pd.DataFrame: X_train, X_test
    """

    orgs_df = (
        column_data.groupby(provider_col)[provider_col]
        .count()
        .sort_values(ascending=False)
    )
    orgs_df = (
        column_data.groupby(provider_col)[provider_col]
        .count()
        .sort_values(ascending=False)
        .reset_index(name="count")
    )
    all_orgs = orgs_df[provider_col].unique()

    # Split orgs to get 'Parent', eg 'ocha-*' -> 'ocha'
    orgs_df["org_parent"] = orgs_df[provider_col].str.split("-").str[0]

    # Display, sorted by provider_col
    print("All Orgs ...")
    display(orgs_df.sort_values(by=provider_col))

    # Count occurrences of each 'org_parent'
    org_parent_counts = orgs_df["org_parent"].value_counts().reset_index(name="count")

    # Filter to keep only those occurring once
    org_parents_single_occurrence = org_parent_counts[org_parent_counts["count"] == 1]

    # Get the 'org_parent' values that occur only once
    single_occurrence_org_parents = org_parents_single_occurrence["org_parent"].tolist()

    # Filter the original DataFrame to keep rows where 'org_parent' occurs only once
    org_parents_unique = orgs_df[
        orgs_df["org_parent"].isin(single_occurrence_org_parents)
    ]

    print("\nOrgs which don't seem to be subsidiaries ...\n")
    display(org_parents_unique)

    single_entities = list(org_parents_unique[provider_col].unique())

    # Remove 'hdx' from single_entities, not good for testing as it's the folks that made HXL! Also some monolithic orgs with very similar data
    single_entities = [
        x for x in single_entities if not x in ["hdx", "ourairports", "un-ocha"]
    ]

    single_entities.sort()

    # Sample single-subsid orgs
    sample_size = int(column_data.shape[0] * test_size) - 1

    # Sample orgs and total rows they have to arrive at fraction specified for test
    total_count = 0
    X_test_orgs = []
    np.random.seed(random_state)
    while total_count < sample_size:
        org = np.random.choice(single_entities, 1)[0]
        count = orgs_df[orgs_df[provider_col] == org]["count"].values[0]
        # Don't allow large orgs to dominate test data, so we get good test coverage
        if count < sample_size / 4:
            total_count += orgs_df[orgs_df[provider_col] == org]["count"].values[0]
            X_test_orgs.append(org)

    X_train_orgs = list(set(all_orgs) - set(X_test_orgs))

    print(f"Train orgs: {X_train_orgs}")
    print(f"Test orgs: {X_test_orgs}")

    # Display orgs_df filtered by X_test_orgs and sorted by count descending
    print("These are the test orgs and their data counts ...")
    display(
        orgs_df[orgs_df[provider_col].isin(X_test_orgs)].sort_values(
            by="count", ascending=False
        )
    )

    # Extract column rows for datasets in X_train_datasets
    X_train = column_data[column_data[provider_col].isin(X_train_orgs)]

    # Extract column rows for datasets in X_test_datasets
    X_test = column_data[~column_data[provider_col].isin(X_train_orgs)]

    train_orgs = X_train[provider_col].unique()
    train_orgs.sort()
    test_orgs = X_test[provider_col].unique()
    test_orgs.sort()

    print(f"\nTrain orgs: {train_orgs}")
    print(f"\nTest orgs: {test_orgs}")

    print(f"\nTrain column data: {X_train.shape}")
    print(f"Test column data: {X_test.shape}")

    return X_train, X_test


X_train, X_test = split_data(data, "Data provider", test_size=0.2, random_state=42)

# Add a column in 'data' called test_train to indicate if row is test or train
data["test_train"] = "train"
data.loc[data["HDX resource id"].isin(X_test["HDX resource id"]), "test_train"] = "test"

# Save data
data.to_csv(f"{LOCAL_DATA_DIR}/hxl_hash_resources_data.csv", index=False)

All Orgs ...


,Data provider,count,org_parent
62,awsd,8,awsd
100,blavatnik-school-of-government-university-of-oxford,3,blavatnik
25,brcmapsteam,31,brcmapsteam
55,cbes,10,cbes
35,cerf,21,cerf
70,cesvi,6,cesvi
89,cfp-rco-nepal,4,cfp
94,cimp,3,cimp
75,cirrolytix,5,cirrolytix
30,clear,26,clear



Orgs which don't seem to be subsidiaries ...



,Data provider,count,org_parent
1,immap,221,immap
2,hdx,135,hdx
5,hera-humanitarian-emergency-response-africa,100,hera
11,redhum,69,redhum
13,fieldsdata,53,fieldsdata
17,ifrc,49,ifrc
20,wfp,41,wfp
25,brcmapsteam,31,brcmapsteam
28,dhs,28,dhs
29,standby-task-force,28,standby


Train orgs: ['ocha-roap', 'cesvi', 'ocha-ds', 'ocha-afghanistan', 'cerf', 'unicef-rdc', 'standby-task-force', 'ocha-cameroon', 'international-organization-for-migration', 'ocha-myanmar', 'unicef-esaro', 'ocha-nigeria', 'ocha-car', 'ewipa', 'fao', 'water-point-data-exchange', 'ocha-indonesia', 'ocha-haiti', 'ocha-colombia', 'ocha-yemen', 'ocha-sudan', 'international-displacement-monitoring-centre-idmc', 'ocha-nepal', 'ocha-rowca', 'immap', 'ocha-pakistan', 'jhucsse', 'somalia-ngo-consortium', 'crs-waro', 'soswcaf', 'kenya-national-bureau-of-statistics', 'ocha-rolac', 'dalberg', 'ocha-chad', 'awsd', 'ocha-rosea', 'ocha-somalia', 'ocha-niger', 'ocha-turkey', 'eth-zurich-weather-and-climate-risks', 'sadc_rvaa', 'unhcr', 'interaction', 'ocha-fiss', 'ocha-opt', 'ocha-ukraine', 'unhcr-afghanistan', 'unicef-data', 'libya-ingo-forum', 'ocha-iraq', 'cirrolytix', 'ocha-burundi', 'ocha-mali', 'cfp-rco-nepal', 'ocha-rosc', 'global-shelter-cluster', 'cbes', 'ocha-libya', 'clear', 'ocha-fts', 'inter-

,Data provider,count,org_parent
5,hera-humanitarian-emergency-response-africa,100,hera
11,redhum,69,redhum
13,fieldsdata,53,fieldsdata
17,ifrc,49,ifrc
25,brcmapsteam,31,brcmapsteam
28,dhs,28,dhs
37,insecurity-insight,17,insecurity
47,education-cluster-yemen,13,education
48,iati,12,iati
52,hrrp-nepal,11,hrrp



Train orgs: ['awsd' 'cbes' 'cerf' 'cesvi' 'cfp-rco-nepal' 'cirrolytix' 'clear'
 'crs-waro' 'dalberg' 'eth-zurich-weather-and-climate-risks' 'ewipa' 'fao'
 'fao-reowa' 'global-shelter-cluster' 'hdx' 'idi' 'immap'
 'inter-sector-coordination-group' 'interaction'
 'international-displacement-monitoring-centre-idmc'
 'international-organization-for-migration' 'ipc' 'ipc-cluster-guinea'
 'jhucsse' 'kenya-national-bureau-of-statistics' 'libya-ingo-forum'
 'ocha-afghanistan' 'ocha-burkina' 'ocha-burundi' 'ocha-cameroon'
 'ocha-car' 'ocha-chad' 'ocha-colombia' 'ocha-dr-congo' 'ocha-ds'
 'ocha-eritrea' 'ocha-ethiopia' 'ocha-fiss' 'ocha-fts' 'ocha-haiti'
 'ocha-indonesia' 'ocha-iraq' 'ocha-libya' 'ocha-mali'
 'ocha-mozambique-hat' 'ocha-myanmar' 'ocha-nepal' 'ocha-niger'
 'ocha-nigeria' 'ocha-opt' 'ocha-pakistan' 'ocha-philippines' 'ocha-roap'
 'ocha-rolac' 'ocha-rosc' 'ocha-rosea' 'ocha-rowca' 'ocha-somalia'
 'ocha-south-sudan' 'ocha-sudan' 'ocha-turkey' 'ocha-ukraine' 'ocha-yemen'
 'ourairpor

### Create LLM fine-tuning prompt files

In [70]:
def create_prompt_file(X_train, prompt_col, filename):
    """
    Create a prompt file from a DataFrame.

    Args:
        X_train (pd.DataFrame): The DataFrame containing the prompts.
        prompt_col (str): The name of the column containing the prompts.
        filename (str): The name of the file to write the prompts to.
    """

    with open(filename, "w") as f:
        for index, row in X_train.iterrows():
            f.write(row[prompt_col] + "\n")

    print(f"Prompts written to {filename}")


def generate_chat_prompt(
    dataset_name,
    resource_name,
    column_name,
    excerpt,
    hxl_tag=None,
    dataset_description="",
    add_response=True,
):
    """
    Generate a chat (eg for GPT-3.5-Turbo) fine tuning prompt for HXL tags given dataset, resource, column information.

    Parameters:
    - dataset_name (str): Name of the dataset.
    - resource_name (str): Name of the resource.
    - column_name (str): Name of the column.
    - excerpt (str): Examples or excerpt of the column.
    - hxl_tag (str, optional): HXL tags for the column. Default is None.
    - dataset_description (str, optional): Description of the dataset. Default is an empty string.
    - add_response (bool, optional): Whether to include the response in the prompt. Default is True.

    Returns:
    - dict: A dictionary containing the prompt and optional completion/response.
    """

    system_message = """
        You are an assistant that replies with HXL tags and attributes"
    """

    resource_name = resource_name.replace(f"{LOCAL_DATA_DIR}/", "")

    column_details = (
        f"resource_name='{resource_name}'; "
        + f"dataset_description='{dataset_description}'; "
        + f"column_name:'{column_name}'; examples: {excerpt}"
    )

    user_prompt = f"What are the HXL tags and attributes for a column with these details? {column_details}"

    prompt = {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt},
        ]
    }

    if add_response:
        prompt["messages"].append({"role": "assistant", "content": hxl_tag})

    # prompt = json.dumps(prompt, index=4)

    return prompt


def generate_prompts(
    df,
    heading_col="Text header",
    resource_name_col="File",
    tag_col="Hashtag with Attributes",
    excerpt_col="Data excerpt",
    datset_description_col="Data description",
    hxl_tag=False,
):
    """
    Generate a set of prompts for HXL tags from a DataFrame.

    Parameters:
    - df (DataFrame): Input DataFrame containing dataset, resource, column information.
    - hxl_tag (bool, optional): Whether to include HXL tags in the prompts. Default is False.
    - heading_col (str, optional): Name of the column containing column headers. Default is 'Text header'.
    - resource_name_col (str, optional): Name of the column containing resource names. Default is 'File'.
    - tag_col (str, optional): Name of the column containing HXL tags. Default is 'Hashtag with Attributes'.
    - excerpt_col (str, optional): Name of the column containing column data excerpts. Default is 'Data excerpt'.
    - dataset_secription_col (str, optional): Name of the column containing dataset descriptions. Default is 'Data description'.

    Returns:
    - str: A string containing JSON-formatted prompts.
    """

    prompts = []
    for index, row in df.iterrows():
        if row["Data description"] is None:
            print("No dataset description, skipping ...")
            continue

        prompt = generate_chat_prompt(
            "",  # Dataset name
            row[resource_name_col],
            row[heading_col],
            row[excerpt_col],
            hxl_tag=row[tag_col],
            dataset_description=row[datset_description_col],
            add_response=hxl_tag,
        )

        prompts.append(prompt)

    return prompts


def save_all_prompts(data, dataset_type):
    """
    Save all prompts generated from the given data to a JSONL file.

    Args:
        data (list): The input data to generate prompts from.
        dataset_type (str): The type of dataset being processed.

    Returns:
        None
    """
    prompt_file = f"{LOCAL_DATA_DIR}/hxl_chat_prompts_{dataset_type}.jsonl"
    prompts = generate_prompts(data, hxl_tag=True)

    print(f"\n\nSaving {len(prompts)} prompts to {prompt_file} ...")

    with open(prompt_file, "w") as f:
        for prompt in prompts:
            f.write(json.dumps(prompt) + "\n")

    print(f"Prompts written to {prompt_file}")


save_all_prompts(X_train, "train")
save_all_prompts(X_test, "test")



Saving 2883 prompts to /content/drive/MyDrive/Colab/hxl-metadata-prediction/data//hxl_chat_prompts_train.jsonl ...
Prompts written to /content/drive/MyDrive/Colab/hxl-metadata-prediction/data//hxl_chat_prompts_train.jsonl


Saving 458 prompts to /content/drive/MyDrive/Colab/hxl-metadata-prediction/data//hxl_chat_prompts_test.jsonl ...
Prompts written to /content/drive/MyDrive/Colab/hxl-metadata-prediction/data//hxl_chat_prompts_test.jsonl


In [71]:
for dataset_type in ["train", "test"]:
    print(f"\n\nExcerpt from {dataset_type} prompt file:\n\n")
    prompt_file = f"{LOCAL_DATA_DIR}/hxl_chat_prompts_{dataset_type}.jsonl"
    with open(prompt_file) as f:
        prompts = [json.loads(line) for line in f]
        for p in prompts[0:10]:
            print(p)



Excerpt from train prompt file:


{'messages': [{'role': 'system', 'content': '\n        You are an assistant that replies with HXL tags and attributes"\n    '}, {'role': 'user', 'content': 'What are the HXL tags and attributes for a column with these details? resource_name=\'/content/drive/MyDrive/Colab/hxl-metadata-prediction/data/DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx\'; dataset_description=\'The dataset from the file "DRC - Baseline Assessment - M23 Crisis 13 - February 20248.xlsx" contains information on the total number of internally displaced persons (IDPs) and returnees in the Democratic Republic of Congo. The data includes the total number of IDP households, individual IDPs, male and female IDPs, and returnees. For instance, there are 319,283 IDP households, 1,548,732 individual IDPs, 646,805 male IDPs, 901,927 female IDPs, and 587,705 returnees.\'; column_name:\'Total IDP HH\'; examples: [319283]'}, {'role': 'assistant', 'content': '#affected+hh'}]}